In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.models import load_model
model = load_model('best_model.h5')

In [3]:
import pandas as pd
import numpy as np

In [4]:
data_path='testing_set_filtered.csv'
data=pd.read_csv(data_path)

In [5]:
data.head()

,Unnamed: 0,ExecutionTime,ID,high,low,close,volume,TimeDiff
0,463,2024-01-11 16:15:00+01:00,Fri00Q1,99.94,89.94,93.50,6.600,NaN
1,464,2024-01-11 16:30:00+01:00,Fri00Q1,99.94,89.94,93.50,0.000,0 days 00:15:00
2,465,2024-01-11 16:45:00+01:00,Fri00Q1,94.75,94.20,94.75,1.225,0 days 00:15:00
3,466,2024-01-11 17:00:00+01:00,Fri00Q1,96.25,96.25,96.25,0.125,0 days 00:15:00
4,467,2024-01-11 17:15:00+01:00,Fri00Q1,93.50,93.50,93.50,0.025,0 days 00:15:00


In [6]:
# Drop unnecessary columns
data =data [['ExecutionTime','high','low','close','volume']] 
data.set_index('ExecutionTime',drop=True,inplace=True) 
data.head()

,high,low,close,volume
ExecutionTime,,,,
2024-01-11 16:15:00+01:00,99.94,89.94,93.50,6.600
2024-01-11 16:30:00+01:00,99.94,89.94,93.50,0.000
2024-01-11 16:45:00+01:00,94.75,94.20,94.75,1.225
2024-01-11 17:00:00+01:00,96.25,96.25,96.25,0.125
2024-01-11 17:15:00+01:00,93.50,93.50,93.50,0.025


In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data[data.columns] = scaler.fit_transform(data)

In [8]:
data.shape

(1713779, 4)

In [9]:
context_length = 30  # The number of past steps to consider
prediction_length = 10  # Number of future steps to predict
# Create a sliding window over the time series data to extract past values
sequence_length = context_length# Total past sequence length
# Split the data into sliding windows
past_values = []
future_values = []
# Sliding window with a step size of sequence_length
for i in range(0, len(data) - sequence_length - prediction_length, sequence_length+prediction_length):
    past_window = data[i : i + sequence_length]  # Past values for input
    future_window = data[i + sequence_length : i + sequence_length + prediction_length]  # Future values
    
    past_values.append(past_window)
    future_values.append(future_window)

In [10]:
past_values=np.array(past_values)
future_values=np.array(future_values)

In [11]:
past_values.shape, future_values.shape

((42844, 30, 4), (42844, 10, 4))

In [12]:
future_values_prediction = model.predict(past_values)

In [13]:
future_values_prediction.shape

(42844, 10, 4)

In [14]:
def inverse_transform_3d(scaled_data, scaler):
    # Get the shape of the scaled data
    n_samples, n_timesteps, n_features = scaled_data.shape

    # Reshape the data to 2D (combine samples and timesteps into one dimension)
    scaled_data_2d = scaled_data.reshape(-1, n_features)  # Shape will be (285620, 4) for example

    # Inverse transform the 2D data
    inverse_transformed_2d = scaler.inverse_transform(scaled_data_2d)

    # Reshape the inverse transformed data back to 3D
    inverse_transformed_3d = inverse_transformed_2d.reshape(n_samples, n_timesteps, n_features)

    return inverse_transformed_3d


In [15]:
predicted_value = inverse_transform_3d(future_values_prediction,scaler)
true_value = inverse_transform_3d(future_values,scaler)

In [16]:
def smape(y_true, y_pred):
    # Avoid division by zero by adding a small constant (epsilon) to the denominator
    epsilon = 1e-4
    numerator = np.abs(y_true - y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2 + epsilon
    smape_value = 100 * np.mean(numerator / denominator, axis=1)  # Average across time steps
    return smape_value

# Assuming y_true and y_pred have shape (28562, 10, 4)
# Example: y_true = np.random.rand(28562, 10, 4), y_pred = np.random.rand(28562, 10, 4)
y_true=true_value
y_pred=predicted_value
# Calculate sMAPE for each feature (axis 2)
feature_wise_smape = []
for feature in range(y_true.shape[2]):
    smape_feature = smape(y_true[:, :, feature], y_pred[:, :, feature])
    feature_wise_smape.append(np.mean(smape_feature))  # Average across samples

# Convert to numpy array for easier handling if needed
feature_wise_smape = np.array(feature_wise_smape)

In [17]:
feature_wise_smape

array([174.59381289, 199.69604749, 163.8433481 , 188.6728199 ])